In [43]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt
import time
import pickle
import csv

In [44]:
def initializeWeights(n_in, n_out):
    """
    # initializeWeights return the random weights for Neural Network given the
    # number of node in the input layer and output layer
    # Input:
    # n_in: number of nodes of the input layer
    # n_out: number of nodes of the output layer
       
    # Output: 
    # W: matrix of random initial weights with size (n_out x (n_in + 1))"""

    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W


In [45]:
def sigmoid(z):
    """# Notice that z can be a scalar, a vector or a matrix
    # return the sigmoid of input z"""
    
    #your code here
    sig = 1.0/(1.0 + np.exp(-z))
    return  sig

In [46]:
def preprocess():
    """ Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.
     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
     Some suggestions for preprocessing step:
     - feature selection"""

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    # Split the training sets into two sets of 50000 randomly sampled training examples and 10000 validation examples. 
    # Your code here.

    trn_pre = np.zeros(shape=(50000, 784))
    val_pre = np.zeros(shape=(10000, 784))
    test_pre = np.zeros(shape=(10000, 784))
    trn_label_pre = np.zeros(shape=(50000,))
    val_label_pre = np.zeros(shape=(10000,))
    test_label_pre = np.zeros(shape=(10000,))
    
    trn_length = 0
    val_length = 0
    test_length = 0
    trn_label_length = 0
    val_label_length = 0
    for key in mat:
        if "train" in key:
            label = key[-1] 
            tup_key = mat.get(key)
            sap_key = range(tup_key.shape[0])
            tup_perm = np.random.permutation(sap_key)
            tup_length = len(tup_key)  # get the length of current training set
            tag_length = tup_length - 1000  # defines the number of examples which will be added into the training set

            # ---------------------adding data to training set-------------------------#
            trn_pre[trn_length:trn_length + tag_length] = tup_key[tup_perm[1000:], :]
            trn_length += tag_length

            trn_label_pre[trn_label_length:trn_label_length + tag_length] = label
            trn_label_length += tag_length

            # ---------------------adding data to validation set-------------------------#
            val_pre[val_length:val_length + 1000] = tup_key[tup_perm[0:1000], :]
            val_length += 1000

            val_label_pre[val_label_length:val_label_length + 1000] = label
            val_label_length += 1000

            # ---------------------adding data to test set-------------------------#
        elif "test" in key:
            label = key[-1]
            tup_key = mat.get(key)
            sap_key = range(tup_key.shape[0])
            tup_perm = np.random.permutation(sap_key)
            tup_length = len(tup_key)
            test_label_pre[test_length:test_length + tup_length] = label
            test_pre[test_length:test_length + tup_length] = tup_key[tup_perm]
            test_length += tup_length
            # ---------------------Shuffle,double and normalize-------------------------#
    train_size = range(trn_pre.shape[0])
    train_perm = np.random.permutation(train_size)
    train_data = trn_pre[train_perm]
    train_data = np.double(train_data)
    train_data = train_data / 255.0
    train_label = trn_label_pre[train_perm]

    validation_size = range(val_pre.shape[0])
    vali_perm = np.random.permutation(validation_size)
    validation_data = val_pre[vali_perm]
    validation_data = np.double(validation_data)
    validation_data = validation_data / 255.0
    validation_label = val_label_pre[vali_perm]

    test_size = range(test_pre.shape[0])
    test_perm = np.random.permutation(test_size)
    test_data = test_pre[test_perm]
    test_data = np.double(test_data)
    test_data = test_data / 255.0
    test_label = test_label_pre[test_perm]

    # Feature selection
    #your code here

    npStack = np.vstack(( train_data, validation_data, test_data)) #Create Stack

    allData = np.array( npStack)
    remove = np.all(allData == allData[0,:], axis = 0) #Identify columns which need to be removed
    
    global RemData
    RemData = allData[:,~remove] #Removing the columns which were identifed in the previous step
    

    # print('Selected features', RemData)
    
    train_data = RemData[0:len(train_data),:] #Updating 'training Data'

    lenTrainData = len(train_data)
    lenValidationData = len(validation_data)

    validation_data = RemData[lenTrainData: (lenTrainData + len(validation_data)),:] #Updating 'Validation Data'

    lenTrainData = len(train_data)
    lenValidationData = len(validation_data)
    lenTestData = len(test_data)

    test_data = RemData[( lenTrainData + lenValidationData): (lenTrainData + lenValidationData + lenTestData),:] #Updating 'test Data'

    print('preprocess done')

    return train_data, train_label, validation_data, validation_label, test_data, test_label


In [47]:
def nnObjFunction(params, *args):
    """% nnObjFunction computes the value of objective function (negative log 
    %   likelihood error function with regularization) given the parameters 
    %   of Neural Networks, thetraining data, their corresponding training 
    %   labels and lambda - regularization hyper-parameter.
    % Input:
    % params: vector of weights of 2 matrices w1 (weights of connections from
    %     input layer to hidden layer) and w2 (weights of connections from
    %     hidden layer to output layer) where all of the weights are contained
    %     in a single vector.
    % n_input: number of node in input layer (not include the bias node)
    % n_hidden: number of node in hidden layer (not include the bias node)
    % n_class: number of node in output layer (number of classes in
    %     classification problem
    % training_data: matrix of training data. Each row of this matrix
    %     represents the feature vector of a particular image
    % training_label: the vector of truth label of training images. Each entry
    %     in the vector represents the truth label of its corresponding image.
    % lambda: regularization hyper-parameter. This value is used for fixing the
    %     overfitting problem.
       
    % Output: 
    % obj_val: a scalar value representing value of error function
    % obj_grad: a SINGLE vector of gradient value of error function
    % NOTE: how to compute obj_grad
    % Use backpropagation algorithm to compute the gradient of error function
    % for each weights in weight matrices.
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    % reshape 'params' vector into 2 matrices of weight w1 and w2
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit j in input 
    %     layer to unit i in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit j in hidden 
    %     layer to unit i in output layer."""

    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args

    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))
    obj_val = 0

    # Your code here
    
    CodingSch = np.zeros((training_label.shape[0], 10))

    CodingSch[np.arange(training_label.shape[0], dtype="int"), training_label.astype(int)] = 1
    training_label = CodingSch
    
    #Forward Propagation
    training_data = np.column_stack((np.array(training_data), np.array(np.ones(training_data.shape[0]))))
    
    #Feed Forward
    #From input layer to hidden layers
    sigma1 = sigmoid(np.dot(training_data, w1.T)) #Dot Product of training data and transpose of w1
    sigma1Shape = np.ones(sigma1.shape[0])
    sigma1 = np.column_stack((sigma1, sigma1Shape))
    
    #From hidden layers to output layers
    sigma2 = sigmoid(np.dot(sigma1, w2.T))
    
    myDelta = sigma2 - training_label #Back propagation
    
    gradW2 = np.dot(myDelta.T, sigma1) #Calculate Gradient Descent
    gradW1 = np.delete(np.dot(((1 - sigma1) * sigma1 * (np.dot(myDelta, w2))).T, training_data), n_hidden, 0)
    
    N = training_data.shape[0]

    lamb_2n = (lambdaval / (2 * N))
    sumOfSquares1 = np.sum(np.square(w1))
    sumOfSquares2 = np.sum(np.square(w2))

    #Updating the obj_val 
    obj_val = ((np.sum(-1 * (training_label * np.log(sigma2) + (1 - training_label) * np.log(1 - sigma2)))) / N) + ( lamb_2n *( sumOfSquares1 + sumOfSquares2))
    
    gradW1 = (gradW1 + (lambdaval * w1)) / N
    gradW2 = (gradW2 + (lambdaval * w2)) / N
    
    # Make sure you reshape the gradient matrices to a 1D array. for instance if your gradient matrices are grad_w1 and grad_w2
    # you would use code similar to the one below to create a flat array
    # obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()),0)
    obj_grad = np.array([])
    obj_grad = np.concatenate((gradW1.flatten(), gradW2.flatten()), 0)

    return (obj_val, obj_grad)


In [48]:
def nnPredict(w1, w2, data):
    """% nnPredict predicts the label of data given the parameter w1, w2 of Neural
    % Network.
    % Input:
    % w1: matrix of weights of connections from input layer to hidden layers.
    %     w1(i, j) represents the weight of connection from unit i in input 
    %     layer to unit j in hidden layer.
    % w2: matrix of weights of connections from hidden layer to output layers.
    %     w2(i, j) represents the weight of connection from unit i in input 
    %     layer to unit j in hidden layer.
    % data: matrix of data. Each row of this matrix represents the feature 
    %       vector of a particular image
       
    % Output: 
    % label: a column vector of predicted labels"""

    labels = np.array([])
    # Your code here
    lenData = np.ones( len(data)) #Adding bias of 1 to input vector

    data = np.column_stack( [data, lenData])
    
    sigma1 = np.column_stack( [sigmoid( data.dot( w1.T)), lenData]) #Sigmoid of data and transpose of w1
    sigma2 = sigmoid( sigma1.dot( w2.T)) #Sigmoid of result calculated in previous step and transpose of w2

    labels = np.argmax( sigma2, axis=1) #Max values for predicted labels

    return labels



In [49]:

"""**************Neural Network Script Starts here********************************"""

lambdaval = 0
with open ("nnscript_comp_n_hid_80.csv", "w") as nnscript_file:
    nnscript_writer = csv.writer(nnscript_file)
    nnscript_writer.writerow(["Lamda", "Hidden Units", "Traning Set Accuracy","Validation Set Accuracy","Test Set Accuracy", "Time Taken in Seconds"])
# set the number of nodes in hidden unit (not including bias unit)
    while lambdaval <=70:
        startTime = time.time()
        train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

        #  Train Neural Network
        n_hidden = 80
        # set the number of nodes in input unit (not including bias unit)
        n_input = train_data.shape[1]
        
        print(lambdaval)
        # set the number of nodes in output unit
        n_class = 10

        # initialize the weights into some random matrices
        initial_w1 = initializeWeights(n_input, n_hidden)
        initial_w2 = initializeWeights(n_hidden, n_class)

        # unroll 2 weight matrices into single column vector
        initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

        # set the regularization hyper-parameter
        

        args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)

        # Train Neural Network using fmin_cg or minimize from scipy,optimize module. Check documentation for a working example

        opts = {'maxiter': 50}  # Preferred value.

        nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)

        # In Case you want to use fmin_cg, you may have to split the nnObjectFunction to two functions nnObjFunctionVal
        # and nnObjGradient. Check documentation for this function before you proceed.
        # nn_params, cost = fmin_cg(nnObjFunctionVal, initialWeights, nnObjGradient,args = args, maxiter = 50)


        # Reshape nnParams from 1D vector into w1 and w2 matrices
        w1 = nn_params.x[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
        w2 = nn_params.x[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

        # Test the computed parameters

        predicted_label = nnPredict(w1, w2, train_data)

        # find the accuracy on Training Dataset
        train_set_acc = str(100 * np.mean((predicted_label == train_label).astype(float))) + '%'
        print('\n Training set Accuracy:' + train_set_acc)

        predicted_label = nnPredict(w1, w2, validation_data)

        # find the accuracy on Validation Dataset
        val_set_acc = str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%'
        print('\n Validation set Accuracy:' + val_set_acc)

        predicted_label = nnPredict(w1, w2, test_data)

        # find the accuracy on Validation Dataset
        test_set_acc = str(100 * np.mean((predicted_label == test_label).astype(float))) + '%'
        print('\n Test set Accuracy:' + test_set_acc)

        timeDifference = time.time()-startTime
        print('\n Time Taken: ' + str(timeDifference)+ ' seconds.')
        
        obj = [RemData, n_hidden, w1, w2, lambdaval]
        pickle.dump(obj, open("params.pickle", "wb"))
        nnscript_writer.writerow([lambdaval, n_hidden ,train_set_acc, val_set_acc, test_set_acc, str(timeDifference)])
        lambdaval += 5

preprocess done
0

 Training set Accuracy:95.586%

 Validation set Accuracy:94.87%

 Test set Accuracy:95.13000000000001%

 Time Taken: 73.38084101676941 seconds.
preprocess done
5

 Training set Accuracy:95.724%

 Validation set Accuracy:94.73%

 Test set Accuracy:95.32000000000001%

 Time Taken: 51.61284112930298 seconds.
preprocess done
10

 Training set Accuracy:95.38600000000001%

 Validation set Accuracy:94.69999999999999%

 Test set Accuracy:95.17999999999999%

 Time Taken: 51.707406997680664 seconds.
preprocess done
15

 Training set Accuracy:95.114%

 Validation set Accuracy:95.1%

 Test set Accuracy:94.98%

 Time Taken: 51.02123999595642 seconds.
preprocess done
20

 Training set Accuracy:95.02000000000001%

 Validation set Accuracy:94.24%

 Test set Accuracy:94.69%

 Time Taken: 50.45037913322449 seconds.
preprocess done
25

 Training set Accuracy:94.774%

 Validation set Accuracy:94.05%

 Test set Accuracy:94.38%

 Time Taken: 54.73166489601135 seconds.
preprocess done
30

